In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Ejemplo de Walk-Forward Optimization
import yfinance as yf

In [7]:
df_original = yf.Ticker('NQ=F')
df_original = df_original.history(start='2010-01-01', end='2024-01-01')

### Función de backtest para cada combinación de parámetros


In [8]:
def backtest_strategy(data,sma_1,sma_2):
    df = data.copy()
    df['SMA_1'] = df['Close'].rolling(window=sma_1).mean()
    df['SMA_2'] = df['Close'].rolling(window=sma_2).mean()

    df['Signal'] = 0
    df.loc[df['SMA_1'] > df['SMA_2'], 'Signal'] = 1
    df.loc[df['SMA_1'] < df['SMA_2'], 'Signal'] = -1

    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()

    df.dropna(inplace=True)

    # Retornar el DataFrame con los cálculos
    return df

### Configuración de la optimización walk-forward


In [9]:
train_window = 3 * 252  # 3 años de datos para entrenamiento (252 días de trading por año)
validation_window = 1 * 252  # 1 año de datos para validación
total_window = train_window + validation_window

### Rango de valores para optimización de medias móviles


In [10]:
sma_1_range = range(10, 50, 5)
sma_2_range = range(100, 300, 25)

### Listas para acumular resultados


In [11]:
validation_returns = []
train_returns = []

### Realizar la optimización Walk-Forward


La optimización walk-forward es un método que evalúa el rendimiento de una estrategia financiera ajustando sus parámetros en ventanas móviles de entrenamiento y validación.

- Ventana de entrenamiento: Se utiliza para optimizar los parámetros de la estrategia (por ejemplo, medias móviles SMA).
- Ventana de validación: Sirve para probar la estrategia con los parámetros seleccionados en un período de datos no visto.

Este proceso se repite avanzando (o "caminando") hacia adelante en el tiempo, lo que proporciona una evaluación más realista y robusta de cómo funcionaría la estrategia en el futuro.

El objetivo principal es evitar el sobreajuste (overfitting), es decir, que una estrategia funcione bien solo en los datos históricos usados para entrenarla.



In [14]:
for start in range(0, len(df_original) - total_window, validation_window):
    # Separar los datos en entrenamiento y validación
    train_data = df_original[start:start + train_window]
    validation_data = df_original[start + train_window:start + total_window]

    # Optimización en el conjunto de entrenamiento
    best_params = None
    best_return_train = -np.inf

    for sma_1 in sma_1_range:
        for sma_2 in sma_2_range:
            df_train_result = backtest_strategy(train_data, sma_1, sma_2)
            total_return_train = (df_train_result['Strategy_Returns'] + 1).prod() - 1
            
            if total_return_train > best_return_train:
                best_return_train = total_return_train
                best_params = (sma_1, sma_2)
    
    train_returns.append(best_return_train)

    # Evaluar en el conjunto de validación con los mejores parámetros
    df_validation_result = backtest_strategy(validation_data, best_params[0], best_params[1])
    total_return_validation = (df_validation_result['Strategy_Returns'] + 1).prod() - 1
    validation_returns.append(total_return_validation)

    print(f'Periodo {start // validation_window + 1}: Mejores parámetros en entrenamiento: SMA_1: {best_params[0]}, SMA_2: {best_params[1]}')
    print(f'Rendimiento en entrenamiento: {best_return_train:.2%}, Rendimiento en validación: {total_return_validation:.2%}\n')

Periodo 1: Mejores parámetros en entrenamiento: SMA_1: 40, SMA_2: 150
Rendimiento en entrenamiento: 5.04%, Rendimiento en validación: 12.94%

Periodo 2: Mejores parámetros en entrenamiento: SMA_1: 25, SMA_2: 250
Rendimiento en entrenamiento: 45.55%, Rendimiento en validación: -1.68%

Periodo 3: Mejores parámetros en entrenamiento: SMA_1: 10, SMA_2: 250
Rendimiento en entrenamiento: 56.74%, Rendimiento en validación: -2.27%

Periodo 4: Mejores parámetros en entrenamiento: SMA_1: 45, SMA_2: 125
Rendimiento en entrenamiento: 29.82%, Rendimiento en validación: 13.62%

Periodo 5: Mejores parámetros en entrenamiento: SMA_1: 40, SMA_2: 125
Rendimiento en entrenamiento: 5.38%, Rendimiento en validación: 18.25%

Periodo 6: Mejores parámetros en entrenamiento: SMA_1: 40, SMA_2: 250
Rendimiento en entrenamiento: 27.83%, Rendimiento en validación: -1.65%

Periodo 7: Mejores parámetros en entrenamiento: SMA_1: 40, SMA_2: 125
Rendimiento en entrenamiento: 73.02%, Rendimiento en validación: 13.62%

P

### Evaluar el rendimiento total en validación fuera de muestra


In [15]:
print(f'Rendimiento promedio en validación fuera de muestra: {np.mean(validation_returns):.2%}')

Rendimiento promedio en validación fuera de muestra: 7.34%
